# scikit example

In [ ]:
# Author: Jake Vanderplas <jakevdp@cs.washington.edu>
#
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_species_distributions
from sklearn.datasets.species_distributions import construct_grids
from sklearn.neighbors import KernelDensity

# if basemap is available, we'll use it.
# otherwise, we'll improvise later...
try:
    from mpl_toolkits.basemap import Basemap
    basemap = True
except ImportError:
    basemap = False

# Get matrices/arrays of species IDs and locations
data = fetch_species_distributions()
species_names = ['Bradypus Variegatus', 'Microryzomys Minutus']

Xtrain = np.vstack([data['train']['dd lat'],
                    data['train']['dd long']]).T
ytrain = np.array([d.decode('ascii').startswith('micro')
                  for d in data['train']['species']], dtype='int')
Xtrain *= np.pi / 180.  # Convert lat/long to radians

# Set up the data grid for the contour plot
xgrid, ygrid = construct_grids(data)
X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])
land_reference = data.coverages[6][::5, ::5]
land_mask = (land_reference > -9999).ravel()

xy = np.vstack([Y.ravel(), X.ravel()]).T
xy = xy[land_mask]
xy *= np.pi / 180.

# Plot map of South America with distributions of each species
fig = plt.figure()
fig.subplots_adjust(left=0.05, right=0.95, wspace=0.05)

for i in range(2):
    plt.subplot(1, 2, i + 1)

    # construct a kernel density estimate of the distribution
    print(" - computing KDE in spherical coordinates")
    kde = KernelDensity(bandwidth=0.04, metric='haversine',
                        kernel='gaussian', algorithm='ball_tree')
    kde.fit(Xtrain[ytrain == i])

    # evaluate only on the land: -9999 indicates ocean
    Z = -9999 + np.zeros(land_mask.shape[0])
    Z[land_mask] = np.exp(kde.score_samples(xy))
    Z = Z.reshape(X.shape)

    # plot contours of the density
    levels = np.linspace(0, Z.max(), 25)
    plt.contourf(X, Y, Z, levels=levels, cmap=plt.cm.Reds)

    if basemap:
        print(" - plot coastlines using basemap")
        m = Basemap(projection='cyl', llcrnrlat=Y.min(),
                    urcrnrlat=Y.max(), llcrnrlon=X.min(),
                    urcrnrlon=X.max(), resolution='c')
        m.drawcoastlines()
        m.drawcountries()
    else:
        print(" - plot coastlines from coverage")
        plt.contour(X, Y, land_reference,
                    levels=[-9999], colors="k",
                    linestyles="solid")
        plt.xticks([])
        plt.yticks([])

    plt.title(species_names[i])

plt.show()

In [8]:
import pymysql
import redis

#redis connection
r = redis.StrictRedis(host='172.17.0.2', port=6379, db=0)

#connect maria rdb
db = pymysql.connect(
        host='172.17.0.4',
        user='root',
        passwd='1234',
        db='rental',
        charset='utf8mb4'
    )
#設置cursor
cursor = db.cursor()
#cursor.execute('SELECT * FROM typeID;')

In [12]:
cursor.execute("show tables;")
cursor.fetchall()

(('baseEnviro',), ('rentInfo',), ('traffic',), ('typeID',))

In [13]:
cursor.execute("select * from typeID;")
cursor.fetchall()

((101, 'Food'),
 (102, 'Supermarket'),
 (201, 'Bus'),
 (202, 'MRT'),
 (203, 'UBike'),
 (204, 'Train'),
 (205, 'Exam'),
 (206, 'Charger'),
 (207, 'Towing'),
 (301, 'BookStore'),
 (302, 'DepartmentStore'),
 (303, 'Library'),
 (304, 'Museum'),
 (305, 'Theater'),
 (401, 'Park'),
 (402, 'Scenic'),
 (403, 'Sport'),
 (501, 'School'),
 (502, 'Hospital'),
 (503, 'Pharmacy'),
 (504, 'Police'),
 (505, 'FireStation'))

In [14]:
cursor.execute("SELECT ID, lat, lng ,cost FROM baseEnviro where typeID=101;")

34493

In [15]:
datas = cursor.fetchall()
datas[:5]

((1, Decimal('25.05603020034000000'), Decimal('121.54421981156000000'), 300),
 (2, Decimal('25.00261600039000000'), Decimal('121.47226001130000000'), 150),
 (3, Decimal('25.06620501950000000'), Decimal('121.49877100046000000'), 145),
 (4, Decimal('25.02663450912000000'), Decimal('121.55180301432000000'), 745),
 (5, Decimal('25.03951001948000000'), Decimal('121.57673801587000000'), 64))

In [18]:
import numpy as np
import pandas as pd
food_df = pd.DataFrame(list(datas))

In [19]:
food_df.columns = ["ID", "lat"," lng" ,"cost"]
food_df.head()

,ID,lat,lng,cost
0,1,25.05603020034000000,121.54421981156000000,300
1,2,25.00261600039000000,121.47226001130000000,150
2,3,25.06620501950000000,121.49877100046000000,145
3,4,25.02663450912000000,121.55180301432000000,745
4,5,25.03951001948000000,121.57673801587000000,64
